In [18]:
import torch
import pandas as pd
from google.colab import drive
from torch import nn
from torch.utils.data import TensorDataset , DataLoader
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.utils import shuffle
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [19]:
data = pd.read_csv("/content/drive/My Drive/dataset/CO2 Emissions_Canada/CO2 Emissions_Canada.csv")
data = shuffle(data)
for i in data:
  print(data[i].describe())

count     7385
unique      42
top       FORD
freq       628
Name: Make, dtype: object
count          7385
unique         2053
top       F-150 FFV
freq             32
Name: Model, dtype: object
count            7385
unique             16
top       SUV - SMALL
freq             1217
Name: Vehicle Class, dtype: object
count    7385.000000
mean        3.160068
std         1.354170
min         0.900000
25%         2.000000
50%         3.000000
75%         3.700000
max         8.400000
Name: Engine Size(L), dtype: float64
count    7385.000000
mean        5.615030
std         1.828307
min         3.000000
25%         4.000000
50%         6.000000
75%         6.000000
max        16.000000
Name: Cylinders, dtype: float64
count     7385
unique      27
top        AS6
freq      1324
Name: Transmission, dtype: object
count     7385
unique       5
top          X
freq      3637
Name: Fuel Type, dtype: object
count    7385.000000
mean       12.556534
std         3.500274
min         4.200000
25%       

In [20]:
needed_data = data[['Vehicle Class', 
                    'Engine Size(L)',
                    'Cylinders',
                    'Fuel Type',
                    'Fuel Consumption City (L/100 km)',
                    'Fuel Consumption Hwy (L/100 km)',
                    'Fuel Consumption Comb (L/100 km)',
                    'CO2 Emissions(g/km)']]
needed_data.head()

,Vehicle Class,Engine Size(L),Cylinders,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),CO2 Emissions(g/km)
2594,PICKUP TRUCK - STANDARD,2.7,6,X,13.3,10.7,12.3,288
3772,SUV - STANDARD,5.3,8,X,15.2,10.8,13.2,310
1488,COMPACT,2.0,4,E,11.9,8.5,10.4,166
7019,SUV - STANDARD,6.2,8,Z,20.9,13.8,17.7,413
1496,MID-SIZE,2.5,4,X,10.6,7.0,8.9,205


In [21]:
replace_dic = dict()
counter = int(1)
for i in data['Vehicle Class'].unique():
  replace_dic[i] = counter
  counter+=1
counter = int(1)
for i in data['Fuel Type'].unique():
  replace_dic[i] = counter
  counter+=1


replace_dic

{'PICKUP TRUCK - STANDARD': 1,
 'SUV - STANDARD': 2,
 'COMPACT': 3,
 'MID-SIZE': 4,
 'PICKUP TRUCK - SMALL': 5,
 'SUV - SMALL': 6,
 'FULL-SIZE': 7,
 'TWO-SEATER': 8,
 'STATION WAGON - SMALL': 9,
 'MINICOMPACT': 10,
 'SUBCOMPACT': 11,
 'VAN - CARGO': 12,
 'SPECIAL PURPOSE VEHICLE': 13,
 'MINIVAN': 14,
 'VAN - PASSENGER': 15,
 'STATION WAGON - MID-SIZE': 16,
 'X': 1,
 'E': 2,
 'Z': 3,
 'D': 4,
 'N': 5}

In [22]:
needed_data = needed_data.replace(replace_dic)
needed_data.head()

,Vehicle Class,Engine Size(L),Cylinders,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),CO2 Emissions(g/km)
2594,1,2.7,6,1,13.3,10.7,12.3,288
3772,2,5.3,8,1,15.2,10.8,13.2,310
1488,3,2.0,4,2,11.9,8.5,10.4,166
7019,2,6.2,8,3,20.9,13.8,17.7,413
1496,4,2.5,4,1,10.6,7.0,8.9,205


In [23]:
train_data, test_data = train_test_split(needed_data,test_size=0.2,random_state=0)

train_feature_data = train_data[['Vehicle Class', 
                    'Engine Size(L)',
                    'Cylinders',
                    'Fuel Type',
                    'Fuel Consumption City (L/100 km)',
                    'Fuel Consumption Hwy (L/100 km)',
                    'Fuel Consumption Comb (L/100 km)']]
train_labels = train_data['CO2 Emissions(g/km)']

test_feature_data = test_data[['Vehicle Class', 
                    'Engine Size(L)',
                    'Cylinders',
                    'Fuel Type',
                    'Fuel Consumption City (L/100 km)',
                    'Fuel Consumption Hwy (L/100 km)',
                    'Fuel Consumption Comb (L/100 km)']]
test_labels = test_data['CO2 Emissions(g/km)']
train_feature_data.head()

,Vehicle Class,Engine Size(L),Cylinders,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km)
3495,5,2.5,4,1,12.0,9.2,10.7
531,11,3.7,6,3,13.6,9.3,11.7
234,4,2.5,4,1,9.3,6.4,8.0
639,8,5.2,10,3,20.6,13.5,17.4
2813,2,3.0,6,4,11.2,8.4,9.9


In [24]:
train_data_tensor = TensorDataset(torch.tensor(train_feature_data.to_numpy(), dtype=torch.float),torch.tensor(train_labels.to_numpy().reshape((-1,1)),  dtype=torch.float))
test_data_tensor = TensorDataset(torch.tensor(test_feature_data.to_numpy(), dtype=torch.float),torch.tensor(test_labels.to_numpy().reshape((-1,1)),  dtype=torch.float))

In [25]:
btch_size = 128
train_data_loader = DataLoader(train_data_tensor, batch_size=btch_size)
test_data_loader = DataLoader(test_data_tensor, batch_size=btch_size)
for x,y in train_data_loader:
  print(x.shape, y.shape)
  break;

torch.Size([128, 7]) torch.Size([128, 1])


In [26]:
class Co2Net(nn.Module):
  def __init__(self, input_dim, output_dim, hidden_dim):
    super(Co2Net,self).__init__()
    self.first_layer = nn.Linear(input_dim, hidden_dim)
    self.first_act_func = nn.LeakyReLU(negative_slope= 0.3)
    self.out_layer = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    out = self.first_layer(x)
    out = self.first_act_func(out)
    out = self.out_layer(out)
    return out

In [27]:
def train(model, data_loader, loss_fn, optim):
  model.train()
  train_loss = 0
  

  for i, (x,y) in enumerate(data_loader):
    x, y = x.to(device) , y.to(device)

    y_hat = model(x)
    mse = loss_fn(y_hat, y)
    train_loss += mse.item()
    optim.zero_grad()
    mse.backward()
    optim.step()

  num_btch = len(data_loader) 
  train_mse = train_loss / num_btch
  print(f'train RMSE {sqrt(train_mse)}')

In [28]:
def test(model, data_loader, loss_fn):
  model.eval()
  test_loss = 0

  with torch.no_grad():
    for x,y in data_loader:
       x, y = x.to(device) , y.to(device)
       y_hat = model(x)
       mse = loss_fn(y_hat, y)
       test_loss += mse.item()
  num_btch = len(data_loader)
  test_mse = test_loss / num_btch
  print(f'test RMSE {sqrt(test_mse)}')


In [29]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [30]:
input_dim = 7
output_dim = 1
hidden_dim = 15

In [31]:
model = Co2Net(input_dim, output_dim, hidden_dim).to(device)
print(model)

Co2Net(
  (first_layer): Linear(in_features=7, out_features=15, bias=True)
  (first_act_func): LeakyReLU(negative_slope=0.3)
  (out_layer): Linear(in_features=15, out_features=1, bias=True)
)


In [32]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [33]:
epoch = 1000

for i in range(epoch):
  print(f'epoch {i+1}/{epoch}')
  train(model,train_data_loader,loss_function, optimizer)
  test(model,test_data_loader, loss_function)

epoch 1/1000
train RMSE 90.41907486638252
test RMSE 28.859660071655338
epoch 2/1000
train RMSE 23.51762845891423
test RMSE 22.202331342657548
epoch 3/1000
train RMSE 22.07182012420372
test RMSE 21.74705181383415
epoch 4/1000
train RMSE 21.933613880340697
test RMSE 21.560005552330082
epoch 5/1000
train RMSE 21.78469508470602
test RMSE 21.36955587498886
epoch 6/1000
train RMSE 21.646042184807275
test RMSE 21.34706273953853
epoch 7/1000
train RMSE 21.428104433267954
test RMSE 21.53258416887261
epoch 8/1000
train RMSE 21.173441160631327
test RMSE 21.396934394276713
epoch 9/1000
train RMSE 20.977658137444095
test RMSE 21.201726377719115
epoch 10/1000
train RMSE 20.841632109351945
test RMSE 20.911248142299282
epoch 11/1000
train RMSE 20.73540872691453
test RMSE 20.52245973703056
epoch 12/1000
train RMSE 20.681524314205863
test RMSE 20.346165011226624
epoch 13/1000
train RMSE 20.712011991064056
test RMSE 20.27678293838957
epoch 14/1000
train RMSE 20.746468234397895
test RMSE 20.24971529285613